In [1]:
# Import necessary packages

import random
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import math


In [2]:
# Define model structures and functions

class Transformer(nn.Module):

    def __init__(self, 
        input_size :int,
        dec_seq_len :int,
        max_seq_len :int,
        out_seq_len :int,
        dim_val :int,  
        n_encoder_layers :int,
        n_decoder_layers :int,
        n_heads :int,
        dropout_encoder,
        dropout_decoder,
        dropout_pos_enc,
        dim_feedforward_encoder :int,
        dim_feedforward_decoder :int,
        dim_feedforward_projecter :int,
        num_var: int=3
        ): 

        """
        Args:
            input_size: int, number of input variables. 1 if univariate.
            dec_seq_len: int, the length of the input sequence fed to the decoder
            max_seq_len: int, length of the longest sequence the model will receive. Used in positional encoding. 
            out_seq_len: int, the length of the model's output (i.e. the target sequence length)
            dim_val: int, aka d_model. All sub-layers in the model produce outputs of dimension dim_val
            n_encoder_layers: int, number of stacked encoder layers in the encoder
            n_decoder_layers: int, number of stacked encoder layers in the decoder
            n_heads: int, the number of attention heads (aka parallel attention layers)
            dropout_encoder: float, the dropout rate of the encoder
            dropout_decoder: float, the dropout rate of the decoder
            dropout_pos_enc: float, the dropout rate of the positional encoder
            dim_feedforward_encoder: int, number of neurons in the linear layer of the encoder
            dim_feedforward_decoder: int, number of neurons in the linear layer of the decoder
        """

        super().__init__() 

        self.dec_seq_len = dec_seq_len
        self.n_heads = n_heads
        self.out_seq_len = out_seq_len
        self.dim_val = dim_val

        self.encoder_input_layer = nn.Sequential(
            nn.Linear(input_size, dim_val),
            nn.Tanh(),
            nn.Linear(dim_val, dim_val))

        self.decoder_input_layer = nn.Sequential(
            nn.Linear(input_size, dim_val),
            nn.Tanh(),
            nn.Linear(dim_val, dim_val))

        self.linear_mapping = nn.Sequential(
            nn.Linear(dim_val, dim_val),
            nn.Tanh(),
            nn.Linear(dim_val, input_size))

        self.positional_encoding_layer = PositionalEncoder(d_model=dim_val, dropout=dropout_pos_enc, max_len=max_seq_len)
        
        self.projector = nn.Sequential(
            nn.Linear(dim_val + num_var, dim_feedforward_projecter),
            nn.Tanh(),
            nn.Linear(dim_feedforward_projecter, dim_feedforward_projecter),
            nn.Tanh(),
            nn.Linear(dim_feedforward_projecter, dim_val))

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_val, 
            nhead=n_heads,
            dim_feedforward=dim_feedforward_encoder,
            dropout=dropout_encoder,
            activation="relu",
            batch_first=True
            )

        self.encoder = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=n_encoder_layers, norm=None)

        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=dim_val,
            nhead=n_heads,
            dim_feedforward=dim_feedforward_decoder,
            dropout=dropout_decoder,
            activation="relu",
            batch_first=True
            )

        self.decoder = nn.TransformerDecoder(decoder_layer=self.decoder_layer, num_layers=n_decoder_layers, norm=None)

    def forward(self, src: Tensor, tgt: Tensor, var: Tensor, device) -> Tensor:
        """
        Args:
            src: the encoder's output sequence. Shape: (S, E) for unbatched input, 
                 (S, N, E) if batch_first=False or (N, S, E) if 
                 batch_first=True, where S is the source sequence length, 
                 N is the batch size, and E is the feature number
            tgt: the sequence to the decoder. Shape: (T, E) for unbatched input, 
                 (T, N, E) if batch_first=False or (N, T, E) if 
                 batch_first=True, where T is the target sequence length, 
                 N is the batch size, E is the feature number.
            src_mask: the mask for the src sequence to prevent the model from 
                      using data points from the target sequence
            tgt_mask: the mask for the tgt sequence to prevent the model from
                      using data points from the target sequence
        """

        src = self.encoder_input_layer(src)

        src = self.positional_encoding_layer(src)

        src = self.encoder(src)

        enc_seq_len = 128

        var = var.unsqueeze(1).repeat(1,enc_seq_len,1)

        src = self.projector(torch.cat([src,var],dim=2))

        tgt = self.decoder_input_layer(tgt)

        tgt = self.positional_encoding_layer(tgt)

        batch_size = src.size()[0]

        # src_mask = generate_square_subsequent_mask(
        # dim1=batch_size*self.n_heads,
        # dim2=self.out_seq_len,
        # dim3=self.out_seq_len-1#enc_seq_len
        # ).to(device)

        # src_mask = generate_square_subsequent_mask(sz1=self.out_seq_len, sz2=enc_seq_len).to(device)

        tgt_mask = generate_square_subsequent_mask(sz1=self.out_seq_len, sz2=self.out_seq_len).to(device)

        output = self.decoder(
            tgt=tgt,
            memory=src,
            tgt_mask=tgt_mask,
            memory_mask=None
            )
                
        output= self.linear_mapping(output)

        return output

class PositionalEncoder(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(1)]
        return self.dropout(x)

def generate_square_subsequent_mask(sz1: int, sz2: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz1, sz2) * float('-inf'), diagonal=1)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [3]:
# Load the dataset

def load_dataset(data_length=128):
    
    # Load .json Files
    with open('/content/Dataset_sine.json','r') as load_f:
        DATA = json.load(load_f)
    B = DATA['B_Field']
    B = np.array(B)
    Freq = DATA['Frequency']
    Freq = np.log10(Freq)  # logarithm, optional
    Temp = DATA['Temperature']
    Temp = np.array(Temp)      
    Hdc = DATA['Hdc']
    Hdc = np.array(Hdc)       
    H = DATA['H_Field']
    H = np.array(H)

    # Format data into tensors
    in_B = torch.from_numpy(B).float().view(-1, data_length, 1)
    in_F = torch.from_numpy(Freq).float().view(-1, 1)
    in_T = torch.from_numpy(Temp).float().view(-1, 1)
    in_D = torch.from_numpy(Hdc).float().view(-1, 1)
    out_H = torch.from_numpy(H).float().view(-1, data_length, 1)

    # Normalize
    in_B = (in_B-torch.mean(in_B))/torch.std(in_B)
    in_F = (in_F-torch.mean(in_F))/torch.std(in_F)
    in_T = (in_T-torch.mean(in_T))/torch.std(in_T)
    in_D = (in_D-torch.mean(in_D))/torch.std(in_D)
    out_H = (out_H-torch.mean(out_H))/torch.std(out_H)

    normH = [torch.mean(out_H),torch.std(out_H)]

    # Attach the starting token and add the noise
    head = 0.1 * torch.ones(out_H.size()[0],1,out_H.size()[2])
    out_H_head = torch.cat((head,out_H), dim=1)
    out_H = out_H_head
    out_H_head = out_H_head + (torch.rand(out_H_head.size())-0.5)*0.1

    print(in_B.size())
    print(in_F.size())
    print(in_T.size())
    print(in_D.size())
    print(out_H.size())
    print(out_H_head.size())

    return torch.utils.data.TensorDataset(in_B, in_F, in_T, in_D, out_H, out_H_head), normH


In [5]:
# Config the model training and testing

def main():

    # Reproducibility
    random.seed(1)
    np.random.seed(1)
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Hyperparameters
    NUM_EPOCH = 2000
    BATCH_SIZE = 128
    DECAY_EPOCH = 150
    DECAY_RATIO = 0.9
    LR_INI = 0.004

    # Select GPU as default device
    device = torch.device("cuda")

    # Load dataset
    dataset, normH = load_dataset()

    # Split the dataset
    train_size = int(0.7 * len(dataset))
    valid_size = int(0.2 * len(dataset))
    test_size = len(dataset) - train_size - valid_size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
    kwargs = {'num_workers': 0, 'pin_memory': True, 'pin_memory_device': "cuda"}
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, **kwargs)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, **kwargs)

    # Setup network
    net = Transformer(
      dim_val=24,
      input_size=1, 
      dec_seq_len=129,
      max_seq_len=129,
      out_seq_len=129, 
      n_decoder_layers=1,
      n_encoder_layers=1,
      n_heads=4,
      dropout_encoder=0.0, 
      dropout_decoder=0.0,
      dropout_pos_enc=0.0,
      dim_feedforward_encoder=40,
      dim_feedforward_decoder=40,
      dim_feedforward_projecter=40).to(device)

    # Log the number of parameters
    print("Number of parameters: ", count_parameters(net))

    # Setup optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=LR_INI) 

    # Train the network
    for epoch_i in range(NUM_EPOCH):

        # Train for one epoch
        epoch_train_loss = 0
        net.train()
        optimizer.param_groups[0]['lr'] = LR_INI* (DECAY_RATIO ** (0+ epoch_i // DECAY_EPOCH))

        for in_B, in_F, in_T, in_D, out_H, out_H_head in train_loader:
            optimizer.zero_grad()
            output = net(src=in_B.to(device), tgt=out_H_head.to(device), var=torch.cat((in_F.to(device), in_T.to(device), in_D.to(device)), dim=1), device=device)
            loss = criterion(output[:,:-1,:], out_H.to(device)[:,1:,:])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.25)
            optimizer.step()
            epoch_train_loss += loss.item()

        # Compute validation
        with torch.no_grad():
            net.eval()
            epoch_valid_loss = 0
            for in_B, in_F, in_T, in_D, out_H, out_H_head in valid_loader:
                output = net(src=in_B.to(device), tgt=out_H_head.to(device), var=torch.cat((in_F.to(device), in_T.to(device), in_D.to(device)), dim=1), device=device)
                loss = criterion(output[:,:-1,:], out_H.to(device)[:,1:,:])
                epoch_valid_loss += loss.item()
        
        print(f"Epoch {epoch_i:2d} "
            f"Train {epoch_train_loss / len(train_dataset) * 1e5:.5f} "
            f"Valid {epoch_valid_loss / len(valid_dataset) * 1e5:.5f}")
        
    # Save the model parameters
    torch.save(net.state_dict(), "/content/Model_Transformer.sd")
    print("Training finished! Model is saved!")

    # Test the network
    with torch.no_grad():
        for in_B, in_F, in_T, in_D, out_H, out_H_head in test_loader:

            # Create dummy out_H_head                            
            outputs = torch.zeros(out_H.size()).to(device)
            tgt = (torch.rand(out_H.size())*2-1).to(device)
            tgt[:,0,:] = 0.1*torch.ones(tgt[:,0,:].size())                        

            # Compute inference
            for t in range(1, out_H.size()[1]):   
                outputs = net(src=in_B.to(device),tgt=tgt.to(device),var=torch.cat((in_F.to(device), in_T.to(device), in_D.to(device)), dim=1), device=device)                     
                tgt[:,t,:] = outputs[:,t-1,:]     
            outputs = net(in_B.to(device),tgt.to(device),torch.cat((in_F.to(device), in_T.to(device), in_D.to(device)), dim=1), device=device)

            # Save results
            with open("/content/pred.csv", "a") as f:
                np.savetxt(f, (outputs[:,:-1,:]*normH[1]+normH[0]).squeeze(2).cpu().numpy())
                f.close()
            with open("/content/meas.csv", "a") as f:
                np.savetxt(f, (out_H[:,1:,:]*normH[1]+normH[0]).squeeze(2).cpu().numpy())
                f.close()
            print("Testing finished! Results are saved!")


if __name__ == "__main__":
    main()

torch.Size([3495, 128, 1])
torch.Size([3495, 1])
torch.Size([3495, 1])
torch.Size([3495, 1])
torch.Size([3495, 129, 1])
torch.Size([3495, 129, 1])
Number of parameters:  28481
Epoch  0 Train 261.39128 Valid 76.05996
Epoch  1 Train 39.48671 Valid 19.14655
Epoch  2 Train 14.55471 Valid 12.03329
Epoch  3 Train 30.56771 Valid 17.70212
Epoch  4 Train 13.13164 Valid 12.46921
Epoch  5 Train 11.06711 Valid 10.30110
Epoch  6 Train 9.84602 Valid 10.15027
Epoch  7 Train 9.31123 Valid 9.38027
Epoch  8 Train 8.73119 Valid 9.65500
Epoch  9 Train 8.97243 Valid 8.97064
Epoch 10 Train 8.36112 Valid 8.61865
Epoch 11 Train 7.59549 Valid 7.81819
Epoch 12 Train 6.96785 Valid 6.76963
Epoch 13 Train 6.07059 Valid 5.27928
Epoch 14 Train 5.09470 Valid 5.09040
Epoch 15 Train 4.51462 Valid 4.23700
Epoch 16 Train 4.12481 Valid 4.94005
Epoch 17 Train 3.96537 Valid 4.89796
Epoch 18 Train 3.71244 Valid 3.78681
Epoch 19 Train 3.42419 Valid 2.70072
Epoch 20 Train 2.49224 Valid 2.59375
Epoch 21 Train 2.08220 Valid 2.14